In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #191970;"><b style="color:white;">MAT files</b></h1></center>

"MAT-files are binary MATLAB® files that store workspace variables. Starting with MAT-file Version 4, there are several subsequent versions of MAT-files that support an increasing set of features. MATLAB releases R2006b and later all support all MAT-file versions."

https://www.mathworks.com/help/matlab/import_export/mat-file-versions.html#:~:text=MAT%2Dfiles%20are%20binary%20MATLAB,support%20all%20MAT%2Dfile%20versions.

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQNMtDfmgyRn-Kq8hBkUIMJG0M1aSvuVUH3IA&usqp=CAU)akspython.com

#Special thanks to Monika Suresh and AHernandez1 for showing how to work with .Mat files

Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

ahernandez1  https://www.kaggle.com/code/ahernandez1/american-epilepsy-society-seizure-a-playground

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

from __future__ import print_function
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from keras import backend as K

import random
import numpy as np
import pandas as pd

import scipy.io
from scipy.signal import spectrogram
import matplotlib.pyplot as plt

#Interictal (the time between seizures)

"A long-standing bias suggests that temporal lobe epilepsy (TLE) is the epilepsy syndrome most often complicated by interictal (the time between seizures) cognitive and behavioral problems. This may be true, but the available evidence does not clearly support this view."

"The interictal period comprises more than 99% of most patients’ lives. Interictal cognitive and behavioral disorders profoundly impair the quality of life. These problems are continuous, unlike the seizures, which are intermittent."

"Encompassing a wide spectrum, these disorders often fit awkwardly into neuropsychiatric categories. Even when patients fit into Diagnostic and Statistical Manual of Mental Disorders (4) categories, they often remain untreated because some physicians believe that using medications to treat other problems might lower the seizure threshold."

https://www.epilepsy.com/complications-risks/moods-behavior/interictal-problems

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

interictal_tst = '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_interictal_segment_0001.mat'
preictal_tst = '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_preictal_segment_0001.mat'
interictal_data = scipy.io.loadmat(interictal_tst)
preictal_data = scipy.io.loadmat(preictal_tst)

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

interictal_array = interictal_data['interictal_segment_1'][0][0][0]
preictal_array = preictal_data['preictal_segment_1'][0][0][0]

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

l = list(range(10000))
for i in l[::5000]:
    print('Interictal')
    i_secs = interictal_array[0][i:i+5000]
    i_f, i_t, i_Sxx = spectrogram(i_secs, fs=5000, return_onesided=False)
    i_SS = np.log1p(i_Sxx)
    plt.imshow(i_SS[:] / np.max(i_SS), cmap='Greens')
    plt.show()
    print('Preictal')
    p_secs = preictal_array[0][i:i+5000]
    p_f, p_t, p_Sxx = spectrogram(p_secs, fs=5000, return_onesided=False)
    p_SS = np.log1p(p_Sxx)
    plt.imshow(p_SS[:] / np.max(p_SS), cmap='Greens')
    plt.show()

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn
all_X = []
all_Y = []

types = ['Patient_1_interictal_segment', 'Patient_1_preictal_segment']

for i,typ in enumerate(types):
    # Looking at 18 files for each event for a balanced dataset
    for j in range(18):
        fl = '/kaggle/input/seizure-prediction/Patient_1/Patient_1/{}_{}.mat'.format(typ, str(j + 1).zfill(4))
        data = scipy.io.loadmat(fl)
        k = typ.replace('Patient_1_', '') + '_'
        d_array = data[k + str(j + 1)][0][0][0]
        lst = list(range(3000000))  # 10 minutes
        for m in lst[::5000]:
            # Create a spectrogram every 1 second
            p_secs = d_array[0][m:m+5000]
            p_f, p_t, p_Sxx = spectrogram(p_secs, fs=5000, return_onesided=False)
            p_SS = np.log1p(p_Sxx)
            arr = p_SS[:] / np.max(p_SS)
            all_X.append(arr)
            all_Y.append(i)

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

# Shuffling the data
dataset = list(zip(all_X, all_Y))
random.shuffle(dataset)
all_X,all_Y = zip(*dataset)
print(len(all_X))

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

# Splitting data into train/test, leaving only 600 samples for testing
x_train = np.array(all_X[:21000])
y_train = np.array(all_Y[:21000])
x_test = np.array(all_X[21000:])
y_test = np.array(all_Y[21000:])

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

batch_size = 128
num_classes = 2
epochs = 30
img_rows, img_cols = 256, 22

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

y_train = tf.keras.utils.to_categorical(y_train, num_classes) 
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

model.summary()

In [ ]:
#Code by Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#np.float is deprecated. Just use float (snippet below)

np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#d

In [ ]:
#Code by ahernandez1  https://www.kaggle.com/code/ahernandez1/american-epilepsy-society-seizure-a-playground
#ARMHZJZ user ahernadez

#How does an espectogram of an eeg electrode looks like? Could we use it to make a prediction?

def plotSpectograms(file, electrode=0):
    search_key = "_segment_"       # search key string
    matFile = scipy.io.loadmat(os.path.join(file))
    segment = dict(filter(lambda item: search_key in item[0], matFile.items()))
    segment_list_values = list(segment.values())

    x = np.array(segment_list_values[0][0][0][0][electrode], dtype=float) #Original np.float was deprecated 
    secs = np.array(segment_list_values[0][0][0][1], dtype=float) #Original np.float was deprecated
    Fs = np.array(segment_list_values[0][0][0][2], dtype=float)[0][0] #Original np.float was deprecated
    NFFT = 1024  # the length of the windowing segments
    dt = 1/Fs
    t = np.arange(0.0, 600.0, dt)

    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(15,10))
    ax1.plot(t, x)
    Pxx, freqs, bins, im = ax2.specgram(x, NFFT=NFFT, Fs=Fs, noverlap=100, sides='twosided')
    # The `specgram` method returns 4 objects. They are:
    # - Pxx: the periodogram
    # - freqs: the frequency vector
    # - bins: the centers of the time bins
    # - im: the .image.AxesImage instance representing the data in the plot
    plt.show()
    
file = [
    '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_interictal_segment_0001.mat',
    '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_preictal_segment_0001.mat',
    '/kaggle/input/seizure-prediction/Dog_3/Dog_3/Dog_3_interictal_segment_0005.mat',
    '/kaggle/input/seizure-prediction/Dog_3/Dog_3/Dog_3_preictal_segment_0005.mat'
]
for f in file:
    plotSpectograms(f, electrode=5)

#Declaring time of Death of this Kaggle Notebook: 13:26 - 01/05/2022

In [ ]:
#Code by ahernandez1  https://www.kaggle.com/code/ahernandez1/american-epilepsy-society-seizure-a-playground

#how an eeg looks like.

#Code by https://www.kaggle.com/code/ahernandez1/american-epilepsy-society-seizure-a-playground

# Code in this cell was partially taken from:
# https://matplotlib.org/3.3.1/gallery/specialty_plots/mri_with_eeg.html

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np

file_to_inspect = scipy.io.loadmat('/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_interictal_segment_0001.mat')
# retrieves only the item which contain the data of interese
search_key = '_segment_'
segment = dict(filter(lambda item: search_key in item[0], file_to_inspect.items()))
segment = list(segment.values())
data = segment[0][0][0][0]
num_electrodes = data.shape[0]             # rows (i.e. electrodes) in the data matrix
n_samples = data.shape[1]                  # number of samples on each row (i.e. electrode's samples)
electrode_names = segment[0][0][0][3][0]   # name or labels of the electrodes
t = 10 * np.arange(n_samples) / n_samples

# create the 'figure'
fig = plt.figure("EEG samples",figsize=(30,15))

ticklocs = []
ax = fig.add_subplot(1, 1, 1)
ax.set_xlim(0, 10)
ax.set_xticks(np.arange(10))
data_min = data.min()
data_max = data.max()
dr = (data_max - data_min) * 0.7   #crowd it a bit
y0 = data_min
y1 = (num_electrodes -1) * dr + data_max
ax.set_ylim(y0, y1)

segs = []
for electrode in range(num_electrodes):
    segs.append(np.column_stack((t, data[electrode, :])))
    ticklocs.append(electrode * dr)
    
offsets = np.zeros((num_electrodes, 2), dtype=float)
offsets[:, 1] = ticklocs

lines = LineCollection(segs, offsets=offsets, transOffset=None)
ax.add_collection(lines)

# Set the yticks to use axes coordinates on the y axis
ax.set_yticks(ticklocs)
ax.set_yticklabels(electrode_names)

ax.set_xlabel('Time (minutes)')

plt.show()

#I think I killed my "Patient" above. Time of death of this Notebook: 13:26, 01/05/2022 

#Acknowledgement:

Monika Suresh https://www.kaggle.com/code/m0nika/seizurecnn

ARMHZJZ ahernandez1  https://www.kaggle.com/code/ahernandez1/american-epilepsy-society-seizure-a-playground